In [152]:
import pandas as pd
import csv

## Votos por Município

In [153]:
# Função para abrir os arquivos
def filter_file(input_file, output_file):
    with open(input_file, mode='r', encoding='ISO-8859-1') as infile, \
         open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.DictReader(infile, delimiter=';', quotechar='"')
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()

        for row in reader:
            if row['DS_CARGO'] == 'DEPUTADO FEDERAL':
                writer.writerow(row)

filter_file('./votacao_secao_2022_RS.csv', './votacao_dep_federal_2022_rs.csv')

In [154]:
df = pd.read_csv('./votacao_dep_federal_2022_rs.csv')
df = df.drop(columns=['DT_ELEICAO', 'DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO',  'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'TP_ABRANGENCIA', 'SG_UF', 'SG_UE'])
df.head()

,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
0,RIO GRANDE DO SUL,86932,CHARQUEADAS,50,163,6,DEPUTADO FEDERAL,5088,ALMIRO RODRIGO GEHRAT,1,1031,210001596925,E.M.E.F. PROF. MARIA DE LOURDES FREITAS DE AND...,"R. ALVÍDEA SILVA COSTA, 120"
1,RIO GRANDE DO SUL,89630,VIAMÃO,59,305,6,DEPUTADO FEDERAL,5088,ALMIRO RODRIGO GEHRAT,1,1171,210001596925,EMEF SANTA MARIA,"ESTRADA DOS CUNHA, 820"
2,RIO GRANDE DO SUL,89010,SAPUCAIA DO SUL,108,188,6,DEPUTADO FEDERAL,5088,ALMIRO RODRIGO GEHRAT,2,1031,210001596925,ESCOLA NOSSA SENHORA DE FÁTIMA,RUA CORONEL SERAFIM PEREIRA 417
3,RIO GRANDE DO SUL,85898,CANOAS,66,95,6,DEPUTADO FEDERAL,5088,ALMIRO RODRIGO GEHRAT,1,1112,210001596925,E.E.E.F. PROFESSORA CLOTILDE BATISTA,"RUA CORONEL CAMISÃO, 415"
4,RIO GRANDE DO SUL,89630,VIAMÃO,59,25,6,DEPUTADO FEDERAL,5088,ALMIRO RODRIGO GEHRAT,1,1120,210001596925,COLÉGIO MARISTA GRAÇAS,"AVENIDA SENADOR SALGADO FILHO, 8326"


In [155]:
# Função para carregar o dataset e criar o dicionário
def criar_dicionario_municipios(caminho_arquivo):
    # Carregar o dataset a partir do arquivo CSV
    df = pd.read_csv(caminho_arquivo)

    # Verificar se as colunas 'nm_mun' e 'cd_mun' estão presentes
    if 'Código' not in df.columns or 'Município' not in df.columns:
        raise ValueError("As colunas 'nm_mun' e 'cd_mun' devem estar presentes no dataset.")

    # Criar o dicionário de mapeamento: nome do município -> código do município
    dicionario_municipios = pd.Series(df['Código'].values, index=df['Município'].str.upper().values).to_dict()

    return dicionario_municipios

# Caminho para o arquivo CSV (ajuste conforme necessário)
caminho_arquivo = '/content/rs_municipios.csv'

# Chamar a função e criar o dicionário
dicionario = criar_dicionario_municipios(caminho_arquivo)

In [162]:
agg_df = df.groupby(['CD_MUNICIPIO','NM_MUNICIPIO', 'NM_VOTAVEL', 'SQ_CANDIDATO'], as_index=False)['QT_VOTOS'].sum()
agg_df = agg_df.drop(columns=['CD_MUNICIPIO','SQ_CANDIDATO'])

agg_df['NM_MUNICIPIO'] = agg_df['NM_MUNICIPIO'].apply(lambda x: x.replace('SANT ANA DO LIVRAMENTO', 'SANTANA DO LIVRAMENTO'))
agg_df['NM_MUNICIPIO'] = agg_df['NM_MUNICIPIO'].apply(lambda x: x.replace('WESTFALIA', 'WESTFÁLIA'))

agg_df['cd_mun'] = agg_df['NM_MUNICIPIO'].map(dicionario)
agg_df['cd_mun'] = agg_df['cd_mun'].astype(int)
agg_df['nm_votavel'] = agg_df['NM_VOTAVEL'].str.upper()
agg_df = agg_df.drop(columns=['NM_VOTAVEL'])
agg_df['qt_votos'] = agg_df['QT_VOTOS'].astype(int)
agg_df = agg_df.drop(columns=['QT_VOTOS'])

# tabela com relação municipios x votos
df_votos = agg_df.loc[:, ['cd_mun', 'nm_votavel', 'qt_votos']]
df_votos.head()
df_votos.to_csv('votos_por_municipio.csv', index=False)

## Candidatos e Partidos

In [163]:
# tabela dos candidatos

df_candidatos = pd.read_csv('/content/consulta_cand_2022_RS.csv', encoding='ISO-8859-1', delimiter=';', quotechar='"')


df_candidatos = df_candidatos[df_candidatos['DS_CARGO'] == 'DEPUTADO FEDERAL']
df_candidatos = df_candidatos[['NM_CANDIDATO', 'NR_PARTIDO', 'NM_PARTIDO', 'SG_PARTIDO', 'DS_GENERO', 'DS_ESTADO_CIVIL', 'DS_GRAU_INSTRUCAO', 'DT_NASCIMENTO', 'DS_COR_RACA']]
df_candidatos = df_candidatos.reset_index(drop=True)
df_candidatos = df_candidatos.rename(columns={'NM_CANDIDATO' : 'nome', 'NR_PARTIDO' : 'numero_partido', 'NM_PARTIDO' : 'nome_partido', \
                                              'DS_GENERO' : 'genero', 'DS_GRAU_INSTRUCAO' : 'escolaridade', 'DS_ESTADO_CIVIL' : 'estado_civil', \
                                              'DT_NASCIMENTO' : 'data_nascimento', 'DS_COR_RACA' : 'raca', 'SG_PARTIDO' : 'sigla'})
df_candidatos.to_csv('candidatos_rs_dep_federal_2022.csv', index=False)
df_candidatos

,nome,numero_partido,nome_partido,sigla,genero,estado_civil,escolaridade,data_nascimento,raca
0,CLAUDINO FARIAS MURILLO JUNIOR,22,PARTIDO LIBERAL,PL,MASCULINO,CASADO(A),SUPERIOR COMPLETO,30/07/1977,BRANCA
1,CLENIO FAVIN,51,PATRIOTA,PATRIOTA,MASCULINO,CASADO(A),SUPERIOR COMPLETO,25/11/1972,BRANCA
2,MARCO ANTONIO MOURA DOS SANTOS,19,PODEMOS,PODE,MASCULINO,CASADO(A),SUPERIOR COMPLETO,22/05/1962,BRANCA
3,JOÃO CARLOS GILLI MARTINS,16,PARTIDO SOCIALISTA DOS TRABALHADORES UNIFICADO,PSTU,MASCULINO,SEPARADO(A) JUDICIALMENTE,SUPERIOR COMPLETO,01/10/1948,BRANCA
4,LUIZ OMAR DUARTE DO AMARAL,20,PARTIDO SOCIAL CRISTÃO,PSC,MASCULINO,SOLTEIRO(A),ENSINO MÉDIO COMPLETO,22/03/1956,BRANCA
...,...,...,...,...,...,...,...,...,...
541,NEORILDO JOSE DASSI,40,PARTIDO SOCIALISTA BRASILEIRO,PSB,MASCULINO,SOLTEIRO(A),SUPERIOR COMPLETO,17/03/1974,BRANCA
542,THAIS ALMARA QUINTANA,40,PARTIDO SOCIALISTA BRASILEIRO,PSB,FEMININO,SEPARADO(A) JUDICIALMENTE,ENSINO MÉDIO COMPLETO,26/01/1982,PARDA
543,MARISTELA PANEGALLI,14,PARTIDO TRABALHISTA BRASILEIRO,PTB,FEMININO,DIVORCIADO(A),SUPERIOR COMPLETO,06/04/1972,BRANCA
544,MARIA IGNES GOULART,40,PARTIDO SOCIALISTA BRASILEIRO,PSB,FEMININO,DIVORCIADO(A),ENSINO FUNDAMENTAL COMPLETO,01/08/1963,PRETA


In [164]:
df_partidos = df_candidatos.loc[:, ['nome_partido', 'numero_partido', 'sigla']]
df_partidos = df_partidos.drop_duplicates()
df_partidos = df_partidos.reset_index(drop=True)
df_partidos.to_csv('partidos.csv', index=False)
df_partidos

,nome_partido,numero_partido,sigla
0,PARTIDO LIBERAL,22,PL
1,PATRIOTA,51,PATRIOTA
2,PODEMOS,19,PODE
3,PARTIDO SOCIALISTA DOS TRABALHADORES UNIFICADO,16,PSTU
4,PARTIDO SOCIAL CRISTÃO,20,PSC
5,MOVIMENTO DEMOCRÁTICO BRASILEIRO,15,MDB
6,UNIDADE POPULAR,80,UP
7,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,45,PSDB
8,PARTIDO DEMOCRÁTICO TRABALHISTA,12,PDT
9,SOLIDARIEDADE,77,SOLIDARIEDADE
